

# Using FPGA Inference Accelerator

In This tutorial, like the previous exercises we will use a Single Shot MultiBox Detector (SSD) on a trained mobilenet-ssd* model show Inference using FPGA Accelerator.



    
## Part 1: Optimize a deep-learning model using the Model Optimizer (MO)

In this section, you will use the Model Optimizer to convert a trained model to two Intermediate Representation (IR) files (one .bin and one .xml). The Inference Engine requires this model conversion so that it can use the IR as input and achieve optimum performance on Intel hardware.


### 1. Importing Dependencies

In [ ]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *

Now, using the model downloader tool we will download the required models that are helpful in this exercise. We will use the **MobileNet-SSD** model. Download the model, specifying the name and output directory.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o models


    
### 2. Run the Model Optimizer on the pretrained Caffe* model. This step generates one .xml file and one .bin file and place both files in the tutorial samples directory (located here: /object-detection/)

In [ ]:
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --data_type FP32 --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/object_detection/common/mobilenet-ssd/FP32 --scale 256 --mean_values [127,127,127]
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --data_type FP16 --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/object_detection/common/mobilenet-ssd/FP16 --scale 256 --mean_values [127,127,127]



   Note: Although this tutorial uses Single Shot MultiBox Detector (SSD) on a trained mobilenet-ssd* model, the Inference Engine is compatible with other neural network architectures, such as AlexNet*, GoogleNet*, MxNet* etc.


The Model Optimizer converts a pretrained Caffe* model to make it compatible with the Intel Inference Engine and optimizes it for Intel® architecture. These are the files you would include with your C++ application to apply inference to visual data.

   Note: if you continue to train or make changes to the Caffe* model, you would then need to re-run the Model Optimizer on the updated model.

### 3. Navigate to the tutorial sample model directory and verify creation of the optimized model files (the IR files)

In [ ]:
!ls models/object_detection/common/mobilenet-ssd/FP32/

## Part 2: Use the mobilenet-ssd* model and Inference Engine in an object detection application


### 1. Source your environmental variables

In [ ]:
! /opt/intel/openvino/bin/setupvars.sh
os.environ["VIDEO"] = "cars_1900.mp4"

In [ ]:
!make

We will be using the below cars video in this example to detect the cars from the input video.  

In [ ]:
videoHTML('Sample Video', 
          ['cars_1900.mp4'])


    
### 2. Run the sample application to use the Inference Engine on the test video

#### Create Job Script 

We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.
In the IoT DevCloud, you can do this using the `qsub` command.
We can submit object_detection_job to 5 different types of edge compute nodes simultaneously or just one node at at time.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets use name the job so that it is easier to distinguish between them.
The job file will be executed directly on the edge compute node.

In [ ]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE
ROIFILE=$OUTPUT_FILE/ROIs.txt
OVIDEO=$OUTPUT_FILE/output.mp4

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs - Updated for OpenVINO 2020.1
    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/BSP/a10_1150_sg1/linux64/lib
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R4_PL1_FP11_MobileNet_Clamp.aocx
    export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
fi

if [ "$FP_MODEL" = "FP32" ]; then
    config_file="conf_fp32.txt"
else
    config_file="conf_fp16.txt"
fi

# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
echo $FP_MODEL
./tutorial1 -i cars_1900.mp4 \
            -m models/object_detection/common/mobilenet-ssd/$FP_MODEL/mobilenet-ssd.xml \
            -d $DEVICE \
            -o $OUTPUT_FILE\
            -fr 3000 

# Converting the text output to a video
./ROI_writer -i cars_1900.mp4 \
             -o $OUTPUT_FILE \
             -ROIfile $ROIFILE \
             -l pascal_voc_classes.txt \
             -r 2.0 # output in half res


    
### Part 3: Run the example Intel® Arria® 10 FPGA


In the cell below, The inference workload will run on the IEI Mustang-F100-A10 FPGA card installed in this node.

In [ ]:
print("Submitting a job to an edge compute node with an Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "results/fpga HETERO:FPGA,CPU FP16" -N obj_det_fpga
print(job_id_fpga[0])
#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/fpga', 'v_progress_'+job_id_fpga[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel FPGA)', 
          ['results/fpga/output.mp4'] 
          )

### Part 4 : Run the example on different hardware


#### Submitting to an edge compute node with CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting a job to an edge compute node with an Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te -F "results/core CPU FP32" -N obj_det_core
print(job_id_core[0])
#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/core', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel  Core)', 
          ['results/core/output.mp4'])

#### Submitting to a node with Intel® Core CPU and using the onboard Intel GPU

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.
    
    
Set target hardware as GPU with -d GPU

In [ ]:
print("Submitting a job to an edge compute node with an Intel Core CPU and an Intel GPU...")
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/gpu GPU FP32" -N obj_det_gpu
print(job_id_gpu[0])
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/gpu', 'v_progress_'+job_id_gpu[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel  GPU)', 
          ['results/gpu/output.mp4'])

#### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2

In [ ]:
print("Submitting job to an edge compute node with Intel NCS2...")
#Submit job to the queue
job_id_ncs2 = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "results/ncs2 MYRIAD FP16" -N obj_det_ncs2
print(job_id_ncs2[0])

#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/ncs2', 'v_progress_'+job_id_ncs2[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel  Movidius)', 
          ['results/ncs2/output.mp4'])


## Step 5: Assess Performance

The running time of each inference task is recorded in `results/*/stats.txt`, where the subdirectory name corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('ncs2', 'Intel\nNCS2'),
            ('fpga', ' IEI Mustang\nF100-A10\nFPGA')]
stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/{arch}/stats.txt'.format(arch=arch), a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
        
summaryPlot(stats_list, 'Architecture', 'Time , seconds', 'Inference Engine Processing Time', 'time', 0.5)
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps', 0.5)